In [ ]:
# Even overgestapt naar de Jupyter Notebook-look omdat het opschonen en transformeren daarin veel soepeler gaat
# Transforming the CSV
from classes import read_it, NEW_CSV_PATH
import pandas as pd

df = read_it()

In [ ]:
df = df.loc[df['status']=='PAID'] # alleen de transacties die succesvol zijn

unequal = df.loc[df['price']!=df['worth']] # Even kijken of er verschil is tussen de kolommen. Is er niet, dus droppen
df.drop('worth', axis=1, inplace=True)

# Overige kolommen verwijderen die we voor de huidige analyse niet nodig hebben
df.drop(['order_id', 
         'product_id',
         'status']
         ,axis=1, inplace=True)

In [ ]:
# extra kolommen aanmaken die maand of uur weergeven
df['created_at'] = pd.to_datetime(df['created_at']) 
df['month'] = df['created_at'].dt.month
df['hour'] = df['created_at'].dt.hour
df['minute'] = df['created_at'].dt.minute

# Prijs omrekenen zodat deze in euro's staat i.p.v. centen
df['price'] = df['price']/100

In [ ]:
# Functie die op basis van de tijd (uur) de transactie indeelt in een van de vijf tijdslots
# OCHTEND: voor 12.00 uur (niet inclusief)
# 1E MATINEE: 12.00 - 15.00 uur
# 2E MATINEE: 15.00 - 17.00 uur
# 1E AVOND: 17.00 - 20.00 uur
# 2E AVOND: na 20.00 uur

def tijdslot(hour):
    if hour < 12:
        return 'Ochtend'
    elif 12 <= hour < 15:
        return '1e Matinee'
    elif 15 <= hour < 17:
        return '2e Matinee'
    elif 17 <= hour < 20:
        return '1e Avond'
    else:
        return '2e Avond'

df['tijdslot'] = df['hour'].apply(tijdslot)

In [ ]:
# Alvast een paar kleine analyses uitvoeren

# Welke product-categorie wordt het beste verkocht

category_totals = df.groupby('category')['price'].sum().reset_index()
category_totals.sort_values(by='price', ascending=False, inplace=True)
print(category_totals.head(5))

In [ ]:
# Hoe vaak wordt elk product verkocht, wat is de top 5

bestsellers = df['product_name'].value_counts().reset_index()
bestsellers.columns = ['product_name', 'count']
sorted_bestsellers = bestsellers.sort_values(by='count', ascending=False)
print(sorted_bestsellers.head(5))

In [ ]:
# In welke tijdslot wordt het meest omgezet

best_tijdslot_grouped = df.groupby('tijdslot')['price'].sum().reset_index()
print(best_tijdslot_grouped)

In [ ]:
# Wat zijn de drie meest verkopende producten in het tijdslot met de meeste omzet?

best_tijdslot_index = best_tijdslot_grouped['price'].idxmax()
best_tijdslot = best_tijdslot_grouped.loc[best_tijdslot_index, 'tijdslot']

df_beste_tijdslot = df.loc[df['tijdslot'] == best_tijdslot]
bestsellers_tijdslot = df_beste_tijdslot['product_name'].value_counts().reset_index()
print(bestsellers_tijdslot.head(3))

In [ ]:
# De getransformeerde dataframe opslaan met de nieuwe naam

df.to_csv(NEW_CSV_PATH, index=False)